In [1]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Users/anura/anaconda3/Library/bin/graphviz'
import sys
import networkx as nx
import numpy as np
import random
import pickle
import sys
from numba import jit
import elfi
import sklearn as sk
import scipy
from scipy.spatial import distance
import pandas as pd

In [2]:
seed = 20170530  # this will be separately given to ELFI
np.random.seed(seed)

networktype = 'pref' #pref, smallworld, grid, ER, korea1, korea2, ckm
# nAgents = 4**2
# nAgents = 10**2
# sidelength = int(nAgents**.5) #10^2 = 100, 32^2 = 1024
#haltMin = .6 # minimum % of nodes active
#haltMax = .7 # maximum % of nodes active

# pRewire = .1 # proportion of edges to rewire
# numDatasets = 100 # number of data sets to generate
# saveData = True #save the output to file?


In [3]:
print(nx.__version__)
#parameters of the script
path = '../data/icpsr/DS0001/paluck-edgelist.csv'
edgelist = pd.read_csv(path)
G = nx.from_pandas_dataframe(edgelist, source='ID', target='PEERID')
print(nx.info(G))
print(nx.number_of_nodes(G))
print(f'connected?\t{nx.is_connected(G)}')
print(f'# of connected components:\t{nx.number_connected_components(G)}')

components = nx.connected_components(G)
sglist = list(nx.connected_component_subgraphs(G))

gmat = []
for g in sglist:
    gmat.append(nx.to_numpy_matrix(g, dtype=np.float))


1.11
Name: 
Type: Graph
Number of nodes: 23882
Number of edges: 132054
Average degree:  11.0589
23882
connected?	False
# of connected components:	57


In [4]:
#for i in range(len(sglist)):
#    print('# of nodes in {i}th component is  - ', str(np.mean(gmat[i])))
seeds = pd.read_csv("../data/icpsr/DS0001/paluck-seed.csv")
seeds['ID'] = ((seeds['SCHIDW2'] * 1000) + pd.to_numeric(seeds['ID'], errors='coerce'))

In [5]:
es = pd.read_csv("../data/icpsr/DS0001/paluck-endstate.csv")
es['ID'] = ((es['SCHIDW2'] * 1000) + pd.to_numeric(es['ID'], errors='coerce'))

 /usr/local/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning:Columns (238,242,244,245,246,247,248,249,382) have mixed types.Specify dtype option on import or set low_memory=False.


In [6]:
def testThresh(agents, mn, mx):
    if np.mean(agents) > mx:
        return False
    if np.mean(agents) < mn:
        return False
    if np.mean(agents) <= 0:
        return False
    if np.mean(agents) >= 1:
        return False

    return True


In [7]:
def genNet(n, k=4, pRewire=.1, type='grid'):
  # create net
  if type == 'grid': #wrap the grid
    net = nx.grid_2d_graph(int(n**.5), int(n**.5), periodic=True)
    #net = nx.grid_2d_graph(3, 3, periodic=True)
    #net = nx.grid_2d_graph(2, 2, periodic=False)
    #plotNet(net)
    #print(len(net.edges()))
    #assert(0)
    # rewire
    numRewired = 0
    #while numRewired < (pRewire * nx.number_of_nodes(net)):
    while numRewired < 1:
      tries = 0
      while tries < 100:
        tries = tries + 1
        #print([numRewired, pRewire * nx.number_of_nodes(net), tries])
        v1 = random.choice(net.nodes())
        v2 = random.choice(net.nodes())
        if not( net.has_edge(v1,v2) or v1==v2 or len(net.neighbors(v1)) <= 1): #net.neighbors is sometimes (often?) a blank set, changed so v1 needs 2 nb
          #print net.neighbors(v1)
          break
      v1Neighbors = net.neighbors(v1)
      #print v1Neighbors
      #print v1
      #print v2
      #print(len(net.edges()))
      tobeDeleted = random.choice(v1Neighbors)
      net.remove_edge(v1, tobeDeleted)
      #print(len(net.edges()))
      #print([v1, tobeDeleted, v2])
      net.add_edge(v1, v2)
      numRewired = numRewired + 1
    #plotNet(net)
    #assert(0)
    return net, nx.to_numpy_matrix(net, dtype=np.float)

  elif type == 'smallworld':
    #net = nx.connected_watts_strogatz_graph(n, k, .15)
    net = nx.connected_watts_strogatz_graph(n, k, pRewire)
    return net, nx.to_numpy_matrix(net, dtype=np.float)  
  elif type == 'pref':
    net = nx.barabasi_albert_graph(n, 2)
    return net, nx.to_numpy_matrix(net, dtype=np.float)
  elif type == 'ER':
    #net = nx.erdos_renyi_graph(n, .006)
    targetDegree = 4.
    nEdgesPossible = ((n*n)-n)/2.
    pEdge = (n * targetDegree) / (2. * nEdgesPossible)
    assert(pEdge <= 1)

    # spare networks will likely be disconnected, so try a bunch
    tries = 100
    while tries > 0:
      net = nx.erdos_renyi_graph(n, pEdge)
      if nx.number_connected_components(net) > 1:
        tries = tries - 1
      else:
        break
    return net, nx.to_numpy_matrix(net, dtype=np.float)



In [8]:
def ltProp(agents, adjMatrix, nAgents, avgDegree=0, haltMin=.49, haltMax=.51, rs = None):
    # init some stuff for numba
    thresholds = np.zeros_like(nAgents)
    inp = np.zeros_like(nAgents)
    step = 0
    numNeighbors = np.zeros_like(nAgents)
    prevMean = 0.
    liveEdges = np.zeros_like(adjMatrix)
    pInfect = np.zeros_like(adjMatrix)
    flips = np.zeros_like(adjMatrix)
    
    globalThreshold = .5
    if rs is None:
        thresholds = globalThreshold * np.random.random((1, nAgents))
    else:
        thresholds = globalThreshold * rs.random((1, nAgents))
    
    numNeighbors = np.sum(adjMatrix, axis=0)
    prevMean = -1
    step = 1
    
    while not testThresh(agents, haltMin, haltMax) and (np.mean(agents) > prevMean):
        #while np.mean(agents) > prevMean:
        prevMean = np.mean(agents)
        inp = np.true_divide(np.dot(agents, adjMatrix), numNeighbors)
        agents = np.logical_or(agents, (inp >= thresholds)).astype(int)

        step = step + 1
        
    #print('LT-proportional stopped at step ' + str(step) + ' '+ str(np.mean(agents)))
    return agents

In [9]:
def ltAbs(agents, adjMatrix, nAgents, avgDegree=0, haltMin=.49, haltMax=.51, rs = None):
    # init some stuff for numba
    thresholds = np.zeros_like(nAgents)
    inp = np.zeros_like(nAgents)
    step = 0
    numNeighbors = np.zeros_like(nAgents)
    prevMean = 0.
    liveEdges = np.zeros_like(adjMatrix)
    pInfect = np.zeros_like(adjMatrix)
    flips = np.zeros_like(adjMatrix)

    # this controls the thresholds/pInfects for all contagion types
    globalThreshold = .5
    
    if rs is None:
        thresholds = np.random.randint(low=1, high=round(avgDegree*globalThreshold), size=(1, nAgents))
    else:
        thresholds = rs.randint(low=1, high=round(avgDegree*globalThreshold), size=(1, nAgents))
        
    numNeighbors = np.sum(adjMatrix, axis=0)
    #prevMean = -1
    step = 1
    
    while not testThresh(agents, haltMin, haltMax) and (np.mean(agents) > prevMean):
        prevMean = np.mean(agents)
        inp = np.dot(agents, adjMatrix)
        agents = np.logical_or(agents, (inp >= thresholds)).astype(int)

        step = step + 1
        
    # print('LT-absolute stopped at step ' + str(step) + ' '+ str(np.mean(agents)))

    return agents

In [10]:
def generateGraph(net, adjMatrix, pp, batch_size = 1, random_state=None):
    gg = []
    print(pp)
    while True:
        nAgents = nx.number_of_nodes(net)
        agents = np.zeros((1, nAgents))
        nodes = nx.nodes(net)
        seed_nodes = []
        inx = []
        count = 0
        for n in nodes:
            if n in seeds['ID'].tolist():
                seed_nodes.append(n)
                inx.append(count)
            count += 1
        
        agents[:,inx] = 1

        es_nodes = []
        es_inx = []
        count_es = 0
        num_es = 0
        for n in nodes:
            if n in es['ID'].tolist():
                es_nodes.append(n)
                es_inx.append(count_es)
                num_es += 1
            count_es += 1
    
        p_activation = num_es/count_es
        
        # seed neighbors of seeds
        for i in range(1):
            agents = np.logical_or(agents, np.dot(agents, adjMatrix)).astype(float) #all neighbors

        avgDegree = 2*net.number_of_edges() / float(net.number_of_nodes())

        if pp >= 0.5:
            agents_type = ltProp(agents, adjMatrix, nAgents, avgDegree=avgDegree, haltMin=p_activation-0.05, haltMax=p_activation+0.05, rs = random_state)
        else:
            agents_type = ltAbs(agents, adjMatrix, nAgents, avgDegree=avgDegree, haltMin=p_activation-0.05, haltMax=p_activation+0.05, rs = random_state)

        #if not testThresh(agents_type, haltMin, haltMax):
         #   print('bad data, LTabs1:\t'+str(np.mean(agents_type)))
       #     continue
        gparam = {}
        gparam['init'] = net
        gparam['graph'] = agents_type
        
        gg.append(gparam['graph'])
        if len(gg) >= batch_size:
            break

    return gg[0]



In [11]:
#n,a = getGraphFromEdgelist(path)
net_s = sglist[0]
adjMat_s = gmat[0]

In [12]:
def compileG(ll, batch_size = 1, random_state=None):
    return generateGraph(net_s, adjMat_s, ll[0], batch_size, random_state)


In [13]:
nAgobs = nx.number_of_nodes(net_s)
agobs = np.zeros((1, nAgobs))
nodeobs = nx.nodes(net_s)
es_nodeobs = []
es_inxobs = []
count_esobs = 0
num_esobs = 0

for n in nodeobs:
    if n in es['ID'].tolist():
        es_nodeobs.append(n)
        es_inxobs.append(count_esobs)
        num_esobs += 1
    count_esobs += 1

agobs[:,es_inxobs] = 1

In [14]:
gr_obs = np.matrix(agobs).astype(int)
print(gr_obs)

[[0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1
  0 0 1 0 1 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 1 0 0
  0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 0 1 1 0 1 0 1 0
  1 0 1 0 1 0 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0
  0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 0 1 0 1 0
  1 0 0 1 0 1 0 0 0 1 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0 1 0
  1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1
  0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 0 0 1]]


In [15]:
prop_prob = elfi.Prior(scipy.stats.bernoulli, 0.5)
print(type(prop_prob))

<class 'elfi.model.elfi_model.Prior'>


In [16]:
print(gr_obs.shape[1])

272


In [17]:
# Y = elfi.Simulator(generateGraph, prop_prob, observed = gr_obs)
Y = elfi.Simulator(compileG, prop_prob, observed = gr_obs)

In [18]:
def yMetric(x, i = 0):
    return x[0,i]


In [19]:
ret = []
for i in range(gr_obs.shape[1]):
    st = str(i)
    var_s = ''.join(['s',st])
    ret.append(var_s)
    com = var_s + ' = ' + 'elfi.Summary(yMetric, Y, '+st+')'
    exec(com)
    

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

In [30]:
dcom = "d=elfi.Distance('euclidean'"
# len_run = min(250, gr_obs.shape[1])
for i in range(gr_obs.shape[1]):
        dcom = dcom + ',s' + str(i)
dcom = dcom + ')'
print(dcom)
exec(dcom)
# d = elfi.Distance('euclidean',s7)
# d = elfi.Distance('euclidean',s0, s1, s2, s3, s4, s5, s6)

d=elfi.Distance('euclidean',s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,s37,s38,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49,s50,s51,s52,s53,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63,s64,s65,s66,s67,s68,s69,s70,s71,s72,s73,s74,s75,s76,s77,s78,s79,s80,s81,s82,s83,s84,s85,s86,s87,s88,s89,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99,s100,s101,s102,s103,s104,s105,s106,s107,s108,s109,s110,s111,s112,s113,s114,s115,s116,s117,s118,s119,s120,s121,s122,s123,s124,s125,s126,s127,s128,s129,s130,s131,s132,s133,s134,s135,s136,s137,s138,s139,s140,s141,s142,s143,s144,s145,s146,s147,s148,s149,s150,s151,s152,s153,s154,s155,s156,s157,s158,s159,s160,s161,s162,s163,s164,s165,s166,s167,s168,s169,s170,s171,s172,s173,s174,s175,s176,s177,s178,s179,s180,s181,s182,s183,s184,s185,s186,s187,s188,s189,s190,s191,s192,s193,s194,s195,s196,s197,s198,s199,s200,s201,s202,s203,s204,s205,s206,s207,s208,s209,s210,s211,s212,s213,s214,s215,s2

SyntaxError: more than 255 arguments (<string>, line 1)

In [31]:
d=elfi.Distance('euclidean',s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,s37,s38,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49,s50,s51,s52,s53,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63,s64,s65,s66,s67,s68,s69,s70,s71,s72,s73,s74,s75,s76,s77,s78,s79,s80,s81,s82,s83,s84,s85,s86,s87,s88,s89,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99,s100,s101,s102,s103,s104,s105,s106,s107,s108,s109,s110,s111,s112,s113,s114,s115,s116,s117,s118,s119,s120,s121,s122,s123,s124,s125,s126,s127,s128,s129,s130,s131,s132,s133,s134,s135,s136,s137,s138,s139,s140,s141,s142,s143,s144,s145,s146,s147,s148,s149,s150,s151,s152,s153,s154,s155,s156,s157,s158,s159,s160,s161,s162,s163,s164,s165,s166,s167,s168,s169,s170,s171,s172,s173,s174,s175,s176,s177,s178,s179,s180,s181,s182,s183,s184,s185,s186,s187,s188,s189,s190,s191,s192,s193,s194,s195,s196,s197,s198,s199,s200,s201,s202,s203,s204,s205,s206,s207,s208,s209,s210,s211,s212,s213,s214,s215,s216,s217,s218,s219,s220,s221,s222,s223,s224,s225,s226,s227,s228,s229,s230,s231,s232,s233,s234,s235,s236,s237,s238,s239,s240,s241,s242,s243,s244,s245,s246,s247,s248,s249,s250,s251,s252,s253,s254,s255,s256,s257,s258,s259,s260,s261,s262,s263,s264,s265,s266,s267,s268,s269,s270,s271)


SyntaxError: more than 255 arguments (<ipython-input-31-909a0200ca66>, line 1)

In [21]:
rej = elfi.Rejection(d, batch_size=1, seed=seed)

In [22]:
N = 10
# You can give the sample method a `vis` keyword to see an animation how the prior transforms towards the
# posterior with a decreasing threshold.
%time result = rej.sample(N, quantile=0.1)

0rogress: |--------------------------------------------------| 0.0% Complete
1rogress: |--------------------------------------------------| 1.0% Complete
0rogress: |█-------------------------------------------------| 2.0% Complete
1rogress: |█-------------------------------------------------| 3.0% Complete
0rogress: |██------------------------------------------------| 4.0% Complete
0rogress: |██------------------------------------------------| 5.0% Complete
1rogress: |███-----------------------------------------------| 6.0% Complete
0rogress: |███-----------------------------------------------| 7.0% Complete
0rogress: |████----------------------------------------------| 8.0% Complete
1rogress: |████----------------------------------------------| 9.0% Complete
1rogress: |█████---------------------------------------------| 10.0% Complete
1rogress: |█████---------------------------------------------| 11.0% Complete
1rogress: |██████--------------------------------------------| 12.0% Compl

In [25]:
result.samples['prop_prob'].mean()


0.0

In [26]:
print(result.samples['prop_prob'])

[0 0 0 0 0 0 0 0 0 0]


In [27]:
for gr in gr_obs:
    print(gr.shape[1])

272


In [ ]:
## sequential monte carlo